In [ ]:
import processing as pP
import pandas as pd
import matplotlib.pyplot as pl
import numpy as np
import confusionMatrix as cM
import decisionTree as tree
import pymongo

In [ ]:
#Constants
markers = ['ESAT6', 'CFP10', 'RV1681', 'P24']

In [ ]:
#Leer de base de datos de joinedLabApp
MONGO_URL = 'mongodb://idenmon.zapto.org:888/'
client = pymongo.MongoClient(MONGO_URL)
db = client.prodLaboratorio
cursor_img_totals = db.joinedLabApp.find({})
#Crear dataframe a partir de datos lab
df_joinedLabApp = pd.DataFrame(list(cursor_img_totals))
df_joinedLabApp = df_joinedLabApp.drop_duplicates('qr')
df_joinedLabApp.index = df_joinedLabApp['qr']

In [ ]:
columns_selected = [m + '_DIAG' for m in markers] + ['Control', 'CODIGO']
columns_selected

In [ ]:
df_joinedLabApp = df_joinedLabApp[columns_selected]
df_joinedLabApp.head()

In [ ]:
suerosD = ['601170500100319','601170500100357',
           '601170500100374','601170500100104',
           '601170500100407','601170500100391']

In [ ]:
#Define lista de qrs a usar
#Sueros dañados
qrs = list(df_joinedLabApp.index)
for sD in suerosD:
    try:
        qrs.remove(sD)
    except:
        pass

In [ ]:
#Proceso completo FIND para lista de qrs
listResults = []

qrsCopy = []
for i,qr in enumerate(qrs):
    result = pP.multiPipeline(str(qr))
    print(qr)
    if(not isinstance(result, str)):
        listResults += result
        for r in result:
            qrsCopy.append(qr)
    else:
        print("QR: " + str(qr) + " problem: " + result)
        
df_algoritmos_resultados = pd.DataFrame.from_records(listResults, index = qrsCopy)
df_algoritmos_resultados.head()

In [ ]:
#Join dataframes, resultados esperados vs reales
df_lab_app_alg = df_joinedLabApp.join(df_algoritmos_resultados)
df_lab_app_alg = df_lab_app_alg[df_lab_app_alg.ESAT6_result_area.notnull()]
df_lab_app_alg.head()

#### Estadísticas globales por marcador por algoritmo. DIAG vs result_area vs result_blobs

In [ ]:
pos = 'P'
neg = 'N'
ind = 'I'
def safe_div(x,y):
    if y == 0:
        return 0
    return x / y

In [ ]:
dfExtendido = pd.DataFrame()

In [ ]:
dfExtendido = pd.DataFrame()
dfExtendido = df_lab_app_alg
proteinas =['CFP10',  'ESAT6',  'RV1681',  'P24']
for proteina in proteinas:
    dfExtendido[proteina+'_vsBlob'] = np.where((dfExtendido[proteina+'_DIAG']== pos)&(dfExtendido[proteina+'_result_blobs']==pos), 'TP','')
    dfExtendido[proteina+'_vsBlob'] = np.where((dfExtendido[proteina+'_DIAG']== pos)&(dfExtendido[proteina+'_result_blobs']==neg), 'FN',dfExtendido[proteina+'_vsBlob'])
    dfExtendido[proteina+'_vsBlob'] = np.where((dfExtendido[proteina+'_DIAG']== neg)&(dfExtendido[proteina+'_result_blobs']==neg), 'TN',dfExtendido[proteina+'_vsBlob'])
    dfExtendido[proteina+'_vsBlob'] = np.where((dfExtendido[proteina+'_DIAG']== neg)&(dfExtendido[proteina+'_result_blobs']==pos), 'FP',dfExtendido[proteina+'_vsBlob'])
    dfExtendido[proteina+'_vsBlob'] = np.where((dfExtendido[proteina+'_DIAG']== ind)|(dfExtendido[proteina+'_result_blobs']==ind), 'IND',dfExtendido[proteina+'_vsBlob'])
    dfExtendido[proteina+'_vsBlob'] = np.where((dfExtendido['Control']== ind)|(dfExtendido['Control']==pos), 'INVALID',dfExtendido[proteina+'_vsBlob'])
    dfExtendido[proteina+'_vsArea'] = np.where((dfExtendido[proteina+'_DIAG']== pos)&(dfExtendido[proteina+'_result_area']==pos), 'TP','')
    dfExtendido[proteina+'_vsArea'] = np.where((dfExtendido[proteina+'_DIAG']== pos)&(dfExtendido[proteina+'_result_area']==neg), 'FN',dfExtendido[proteina+'_vsArea'])
    dfExtendido[proteina+'_vsArea'] = np.where((dfExtendido[proteina+'_DIAG']== neg)&(dfExtendido[proteina+'_result_area']==neg), 'TN',dfExtendido[proteina+'_vsArea'])
    dfExtendido[proteina+'_vsArea'] = np.where((dfExtendido[proteina+'_DIAG']== neg)&(dfExtendido[proteina+'_result_area']==pos), 'FP',dfExtendido[proteina+'_vsArea'])
    dfExtendido[proteina+'_vsArea'] = np.where((dfExtendido[proteina+'_DIAG']== ind)|(dfExtendido[proteina+'_result_area']==ind), 'IND',dfExtendido[proteina+'_vsArea'])
    dfExtendido[proteina+'_vsArea'] = np.where((dfExtendido['Control']== ind)|(dfExtendido['Control']==pos), 'INVALID',dfExtendido[proteina+'_vsArea'])
dfExtendido.head()

In [ ]:
dfExtendido.to_csv('../data/ComparativeBlobvsAreas.csv')

In [ ]:
#Blurry 
dfExtendido = dfExtendido[dfExtendido['blurry'] == 0.0]
dfExtendido.head()

In [ ]:
g=globals()
proteinas =['CFP10',  'ESAT6',  'RV1681',  'P24']
for proteina in proteinas:
    g['{0}Counts_vsBlobs'.format(proteina)] = dfExtendido[proteina+'_vsBlob'].value_counts()
    g['{0}Counts_vsAreas'.format(proteina)] = dfExtendido[proteina+'_vsArea'].value_counts()


In [ ]:
versiones = ['Blobs', 'Areas'] #automatizarlo para extraer las versiones del DataFrame
d={}
#for ver, prot in mezclaVersionesProteinas:
for ver in versiones:
    #v = ver.replace('.','')
    for prot in proteinas:
        g['{0}'.format(prot)]={}
        resProt = g['{0}Counts_vs{1}'.format(prot,ver)]
        g['sens_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0) + resProt.get('FN',0)))
        #g['prec_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0)+resProt.get('FP',0)))
        g['specif_{0}'.format(prot)]= safe_div(resProt.get('TN',0), (resProt.get('TN',0)+resProt.get('FP',0)))
        g['posPV_{0}'.format(prot)]= safe_div(resProt.get('TP',0),(resProt.get('TP',0)+resProt.get('FP',0)))
        g['negPV_{0}'.format(prot)]= safe_div(resProt.get('TN',0),(resProt.get('TN',0)+resProt.get('FN',0)))
        g['{0}'.format(prot)].update({'Sensitivity': g['sens_{0}'.format(prot)] })
        #g['{0}'.format(prot)].update({'Prescition': g['prec_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Specificity': g['specif_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Positive Predictive Value': g['posPV_{0}'.format(prot)] })
        g['{0}'.format(prot)].update({'Negative Predictive Value': g['negPV_{0}'.format(prot)] })
        totalPruebas = resProt.get('TP',0)+resProt.get('FP',0)+resProt.get('TN',0)+resProt.get('FN',0)+resProt.get('IND',0)
        g['{0}'.format(prot)].update({'#TotalPruebas': totalPruebas})
        g['{0}'.format(prot)].update({'#TP': resProt.get('TP',0) })
        g['{0}'.format(prot)].update({'#FP': resProt.get('FP',0) })
        g['{0}'.format(prot)].update({'#TN': resProt.get('TN',0) })
        g['{0}'.format(prot)].update({'#FN': resProt.get('FN',0) })
        g['{0}'.format(prot)].update({'#Indet': resProt.get('IND',0)})
        g['{0}'.format(prot)].update({'#Invalidos': resProt.get('INVALID',0)})
        d.update({prot: g['{0}'.format(prot)]})
    g['df{0}'.format(ver)] = pd.DataFrame.from_dict(d, orient='Index')

In [ ]:
dfAreasT = dfAreas.T
dfBlobsT = dfBlobs.T
dfComp = dfBlobsT.join(dfAreasT,lsuffix='__vBlobs', rsuffix='__vAreas')
dfComp

In [ ]:
dfComp.to_csv('../data/StatisticsAlgorithms.csv')